In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import fashion_mnist

# Cargar el conjunto de datos Fashion MNIST
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Expandir el canal para convertir de escala de grises (28x28) a (28, 28, 1)
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# Crear un generador personalizado para redimensionar y convertir imágenes en tiempo real
def custom_image_generator(images, labels, batch_size):
    datagen = ImageDataGenerator(
        rescale=1./255,  # Normalización
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        shear_range=0.1,
        fill_mode='nearest'
    )

    generator = datagen.flow(images, labels, batch_size=batch_size)

    while True:
        batch_images, batch_labels = next(generator)

        # Redimensionar y convertir cada imagen del lote a RGB (224x224)
        batch_images_resized = np.array([
            tf.image.resize(image, (224, 224)) for image in batch_images
        ])

        # Convertir de escala de grises a RGB replicando el canal
        batch_images_rgb = np.repeat(batch_images_resized, 3, axis=-1)

        yield batch_images_rgb, batch_labels

# Definir los generadores para entrenamiento y validación
batch_size = 32
train_generator = custom_image_generator(train_images, train_labels, batch_size)
test_generator = custom_image_generator(test_images, test_labels, batch_size)


In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential

# Cargar el modelo MobileNetV2 preentrenado en ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
base_model.trainable = False

# Construir el modelo completo
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # 10 clases para Fashion MNIST
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Calcular el número de pasos por época
steps_per_epoch = len(train_images) // batch_size
validation_steps = len(test_images) // batch_size

# Entrenar el modelo usando el generador personalizado
model.fit(train_generator, epochs=10, steps_per_epoch=steps_per_epoch, validation_data=test_generator, validation_steps=validation_steps)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 223s 113ms/step - accuracy: 0.7976 - loss: 0.5688 - val_accuracy: 0.8500 - val_loss: 0.3993
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 205s 109ms/step - accuracy: 0.8604 - loss: 0.3740 - val_accuracy: 0.8588 - val_loss: 0.3807
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 191s 102ms/step - accuracy: 0.8695 - loss: 0.3451 - val_accuracy: 0.8730 - val_loss: 0.3494
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 190s 102ms/step - accuracy: 0.8783 - loss: 0.3250 - val_accuracy: 0.8607 - val_loss: 0.3674
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 192s 102ms/step - accuracy: 0.8799 - loss: 0.3196 - val_accuracy: 0.8745 - val_loss: 0.3396
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 194s 104ms/step - accuracy: 0.8840 - loss: 0.3066 - val_accuracy: 0.8765 - val_loss: 0.3349
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 200s 106ms/step - accuracy: 0.8864 - loss: 0.3002 - val_accuracy: 0.8811 - val_loss: 0.3335
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 184s 98ms/step - acc

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,753,760 (10.50 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 330,518 (1.26 MB)

In [5]:
model.save("second_version_cnn.h5")